# Starting with getting the products categories from amazon, making a template url to run on each category.

In [32]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import csv

from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


URL = 'https://www.amazon.com/Best-Sellers/zgbs'
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36','referer': 'https://www.amazon.com/'}
html = requests.get(URL, headers=header)
soup = BeautifulSoup(html.content, 'html.parser')
category = soup("div", attrs={
                'class': '_p13n-zg-nav-tree-all_style_zg-browse-item__1rdKf _p13n-zg-nav-tree-all_style_zg-browse-height-small__nleKL'})


categoryNames = list()


#categoryURL = list()

for name in category:
    categoryNames.append(name.get_text())


# -----------------------------------------------------------------------------------------------------------------------------------------

# template = "https://www.amazon.com/s?k={q}&crid=SQLSKT8LVCGV&sprefix=%2Caps%2C254&ref=nb_sb_ss_recent_1_0_recent"
template = "https://www.amazon.com/s?k={q}&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1"

searchTerms = list()

for i in categoryNames:
    i = i.replace('&', '')
    searchTerms.append(i)



SearchLinks = list()
for i in searchTerms: #we are getting blocked for to many requests so we did it in 2 steps [:29] - [29:]
    SearchLinks.append(template.format(q=i))

for i in SearchLinks:
    print(i)




https://www.amazon.com/s?k=Amazon Devices  Accessories&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Amazon Launchpad&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Appliances&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Apps  Games&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Arts, Crafts  Sewing&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Audible Books  Originals&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Automotive&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Baby&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Beauty  Personal Care&crid=38FTQ2MB4WEA&sprefix=laptop%2Caps%2C323&ref=nb_sb_noss_1
https://www.amazon.com/s?k=Books&crid=38FTQ2MB4

# After we got the urls for each category we iterate the pages in each category and storing the url for each product in page and writing it to a file

In [2]:
#selenium web object extraction get text

def getNextPageUrl(nextPageTag):#TAG
    for i in range(0, len(nextPageTag)):
        return nextPageTag[i].get_attribute('href')

def getItemText(obj):
    for i in range(0, len(obj)):
        return obj[i].text


def getItemLink(obj):
    for i in range(0, len(obj)):
        return obj[i].get_attribute('href')
    



In [ ]:

#opens a webdriver 
driver = webdriver.Chrome(ChromeDriverManager().install())

# used for getting items' urls in each category link (SearchLinks) from previous stage
itemsLinks = list()

for link in SearchLinks:
    driver.get(link)
    time.sleep(2)
    
    #link tag of each item
    LinkTag = driver.find_elements_by_xpath("//a[contains(@class, 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')]")
    # itemsLinks.append(getItemLink(LinkTag))
    for i in range(0, len(LinkTag)):
        itemsLinks.append(LinkTag[i].get_attribute('href'))

    # get next page url
    nextPageTag = driver.find_elements_by_xpath("//a[contains(@class, 's-pagination-next')]")
    while nextPageTag:
        nextPageUrl = getNextPageUrl(nextPageTag)
        driver.get(nextPageUrl)
        time.sleep(2)
        nextPageTag = driver.find_elements_by_xpath("//a[contains(@class, 's-pagination-next')]")
        LinkTag = driver.find_elements_by_xpath("//a[contains(@class, 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')]")
        # itemsLinks.append(getItemLink(LinkTag))
        for i in range(0, len(LinkTag)):
            itemsLinks.append(LinkTag[i].get_attribute('href'))



In [6]:

#writes the items urls to file_name.txt
with open('itemsUrls.txt', 'w') as f:
    for i in itemsLinks:
            # write each item on a new line
        f.write("%s\n" % i)
    print('Done copying urls to a file')

#removing duplicates
itemsLinks = list(dict.fromkeys(itemsLinks))
# print(len(list(dict.fromkeys(itemsLinks))))
print('Total links after removing duplicates ' + str(len(itemsLinks)))


Done copying urls to a file
Total links after removing duplicates 3972


# Opening the urls_to_items file and collecting the features for each item url in file

In [ ]:
# from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
# import requests
import csv
import warnings
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
warnings.filterwarnings('ignore')

#go to each item in itemsUrls file and get it's fields , inserting to lists and making a dataframe

driver = webdriver.Chrome(ChromeDriverManager().install())
#opens the url file
with open("itemLinksLast_all.txt") as file:
    lines = file.readlines()
    lines = [line.rstrip() for line in lines]

# lines = itemsLinks

Name = list()
Brand = list()  # a id=bylineInfo
Rating = list()  # span id="acrCustomerReviewText" /
Comments = list()  # a id=askATFLink > span class="a-size-base"
#AnswersAmount = list() #
# <i class="a-icon a-icon-addon p13n-best-seller-badge">#1 Best Seller</i>
BestSeller = list()
Price = list()  # has to be from last stage
# iCategory = list()  # span class="ac-keyword-link"
#IsFreeShipping = list()  #
Discount = list()  # div class="a-section a-spacing-medium tradein-price-block"
AmazonChoice = list()  # span class="ac-badge-text-primary ac-white"
Stars = list()  # span id=acrPopover

filename = 'items.csv'
#run for each item's url and collect the data

for line in lines[1255:]:#for line in lines[-10700:-5000]:
    driver.get(line)
    
    iNames = driver.find_elements_by_xpath("//span[contains(@id, 'productTitle')]") 
    if iNames:
        Name.append(getItemText(iNames))
    else:
        iNames = 'NaN'
        Name.append(iNames)

    iBrand = driver.find_elements_by_xpath("//a[contains(@id, 'bylineInfo')]")
    if iBrand:
        Brand.append(getItemText(iBrand))
    else:
        iBrand = 'NaN'
        Brand.append(iBrand)

    iRating = driver.find_elements_by_xpath("//span[contains(@id, 'acrCustomerReviewText')]")
    if iRating:
        Rating.append(getItemText(iRating))
    else:
        iRating = 'NaN'
        Rating.append(iRating)

    iBestSeller = driver.find_elements_by_xpath("//i[contains(@class, 'a-icon a-icon-addon p13n-best-seller-badge')]")
    if iBestSeller:
        BestSeller.append(1)
    else:
        
        BestSeller.append(0)

    iAmazonChoice = driver.find_elements_by_xpath("//span[contains(@class, 'ac-badge-text-primary ac-white')]")
    if iAmazonChoice:
        AmazonChoice.append(1)
    else:
        AmazonChoice.append(0)

    iComments = driver.find_elements_by_xpath("//a[contains(@id, 'askATFLink')]")
    if iComments:
        Comments.append(getItemText(iComments))
    else:
        iComments = 'NaN'
        Comments.append(iComments)

    iPrice = driver.find_elements_by_xpath("//span[contains(@class, 'a-price a-text-price a-size-medium apexPriceToPay')]")
    if iPrice:
        Price.append(getItemText(iPrice))
    else:
        Price.append('NaN')

    iStars = driver.find_elements_by_xpath("//span[contains(@class, 'a-size-medium a-color-base')]")
    if iStars:
        Stars.append(getItemText(iStars))
    else:
        iStars = 'NaN'
        Stars.append(iStars)

# Some checking after pulling the features for items and exporting our new dataframe to a csv file.

In [25]:
print(len(Name))
print(len(Brand))
print(len(Rating))
print(len(BestSeller))
print(len(AmazonChoice))
print(len(Comments))
print(len(Price))
print(len(Stars))

7120
7120
7120
7120
7120
7120
7120
7120


In [ ]:
rows = list()

df = pd.DataFrame({'Name': Name, 'Brand': Brand, 'Rating': Rating, 'BestSeller': BestSeller,
                  'AmazonChoice': AmazonChoice, 'Comments': Comments, 'Price': Price, 'Stars': Stars})

df.to_csv('itemLinks-copy.csv', index=False)

df




In [27]:
# for i in rows:
#     print(i)

# with open('2380_dataFrame_new.csv', 'w') as f:
#     # f.write(Details)
#     for i in rows:
#         f.write(i)